In [1]:
from nltk.parse.stanford import StanfordDependencyParser
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES
import spacy
import nltk
from word2number import w2n
from find_similar import *

ps = nltk.PorterStemmer()

/home/guest_aashay/.local/lib/python2.7/site-packages/requests/__init__.py:83: RequestsDependencyWarning: Old version of cryptography ([1, 2, 3]) may cause slowdown.
  warnings.warn(warning, RequestsDependencyWarning)


In [2]:
from gensim.models import Word2Vec,KeyedVectors
wvec = KeyedVectors.load_word2vec_format("../btp2/datasets/GoogleNews-vectors-negative300.bin", binary=True)
fl = open("small_verb.txt")
seedverbs = []
from nltk.corpus import wordnet

seedtype = {}
for line in fl:
    seedtype[line.split()[0]] = line.split()[1]
    line = line.strip().split(" ")[0]
    seedverbs.append(line.split()[0])


In [3]:
class linguistic_operations():
    def __init__(self):
        self.depRelations = []
        self.lastrelation = []
        self.lastGraph = []
        self.premGraph = []
        self.question = ""
        self.ind = {}
        self.visited = []
        self.path_to_jar = "./stanford-parser/stanford-parser.jar"
        self.path_to_models_jar = "./stanford-parser/stanford-parser-3.4.1-models.jar"
        self.dependency_parser = StanfordDependencyParser(path_to_jar = self.path_to_jar, path_to_models_jar = self.path_to_models_jar)
        return
    
    
    def dependencyParse(self,sentence):
        self.question = sentence
        result = self.dependency_parser.raw_parse(sentence)
        dep = result.next()
        self.depRelations = list(dep.triples())
        return self.depRelations
    
    
    def numNoun(self):
        allrelations = {}
        lemmatizer = Lemmatizer(LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES)
        for relation in self.depRelations:
            if relation[1] == 'num':
                seed = ps.stem(relation[0][0])
                if seed not in allrelations:
                    allrelations[seed] = []
#                 print ("rela = ",str(relation[2][0]))
                allrelations[seed].append(w2n.word_to_num(str(relation[2][0])))
        self.allrelations =  allrelations
        return allrelations
    
    
    def findroot(self):
        for relation in self.lastrelation:
            if relation[0][1][0]=='W':
                return ps.stem(relation[0][0])
            elif relation[2][1][0]=='W':
                return ps.stem(relation[2][0])
        return "NAN"
    
    
    def makeGraphLast(self):
        
        last = nltk.sent_tokenize(self.question)[-1]
        allwords = self.makeseedvocab(last)
        result = self.dependency_parser.raw_parse(last)
        dep = result.next()
        self.lastrelation = list(dep.triples())
#         print (self.lastrelation)
        newRelation = []
        for relation in self.lastrelation:
            new = (ps.stem(relation[0][0]),relation[1],ps.stem(relation[2][0]))
            newRelation.append(new)

        
#         print (newRelation)
        cnt =0 
        for token in allwords:
            self.ind[token] = cnt
            cnt+=1
    
        graph = [[] for i in range(len(allwords)) ]
        for relation in newRelation:
            graph[self.ind[relation[0]]].append(self.ind[relation[2]])
            graph[self.ind[relation[2]]].append(self.ind[relation[0]])
        self.visited = [0 for i in range(len(allwords))]
        return graph
            
    
    def dfs(self,root,graph):
        self.visited[root] = 1
        for i in range(len(graph[root])):
            if self.visited[graph[root][i]]==0:
                self.dfs(graph[root][i],graph)
        return 
            
        
    
    
    def makeseedvocab(self,last):
        allwords = set()
        sentences = nltk.word_tokenize(last)
        for word in sentences:
            allwords.add(ps.stem(word))
#         print (allwords)
        return allwords
    
    
    def whoseQuantity(self):
        
        for relation in self.lastrelation:
#             print ps.stem(relation[0][0]),ps.stem(relation[2][0])
            if (relation[0][1] == "NNS" or relation[0][1] == "NN") and ps.stem(relation[0][0]) in self.allrelations:
                return ps.stem(relation[0][0])
            elif (relation[2][1]== "NNS" or relation[2][1]== "NN") and ps.stem(relation[2][0]) in self.allrelations:
                return ps.stem(relation[2][0])
    
    
            

In [4]:
def findtransition(linguistic,x):
        temp = nltk.sent_tokenize(linguistic.question)
#         for i in range 
        ll = nltk.word_tokenize(temp[-1])
        for i in range(len(ll)) :
            if ll[i]=="need" or ll[i]=="needed":
                return '-'
        sentences = temp[0:-1]
        premsentence = ""
        for i in sentences:
            premsentence += i
        result = linguistic.dependency_parser.raw_parse(premsentence)
        dep = result.next()
        deppremise = list(dep.triples())
        for relation in deppremise:
#             print "relation= " ,relation[0][1][0:2],relation[0][1][0:2]
            if relation[0][1][0:2] ==u"VB":
                print "verb",ps.stem(relation[0][0])
                ms = x.most_similar(ps.stem(relation[0][0]),seedverbs,wvec)
                print "seedtype =", ms,seedtype[ms]
                if seedtype[ms] == '+' or seedtype[ms] =='-':
                    return seedtype[ms]
            if relation[2][1][0:2] ==u"VB":
                print "verb",ps.stem(relation[2][0])
                ms = x.most_similar(ps.stem(relation[2][0]),seedverbs,wvec)
                print "seedtype =",ps.stem(relation[0][0]),ms,seedtype[ms]
                if seedtype[ms] == '+' or seedtype[ms] =='-':
                    return seedtype[ms]
        return "0"
def findanswer(allrelations,trans,qentity):
#     print (qentity)
    quantities = allrelations[qentity]
    if len(quantities)==1:
        return int(quantities[0])
    ans =0.0
    if trans == '+' or trans =='0':
        for i in quantities:
            ans+=int(i)
    if trans == '-':
        ans = abs(int(quantities[0])-int(quantities[1]))
    return ans
            
    

In [5]:

def calculate(inp_sent):
    print "\n--------\n"
    linguistic = linguistic_operations()
    x = linguistic.dependencyParse(inp_sent)
    print inp_sent
    allrelation = linguistic.numNoun()
    print allrelation
    graph = linguistic.makeGraphLast()
    qentity = linguistic.whoseQuantity()
#     print ("qentity " ,qentity)
    x = find_similar()
    trans = findtransition(linguistic,x)
    return findanswer(allrelation,trans,qentity)

In [ ]:
qfl = open("questions.txt")
questions = []
for line in qfl:
    ans = calculate(line.split("||")[0].strip())
    if int(ans) == int(line.split("||")[1]):
        print ans,"1"
    else:
        print ans,"0"

In [9]:
print calculate("I have three books and four pens . How many books do I have now?")
print calculate("I have 3 books and 4 pens. I gave 1 book to Tom. How many books do I have now?")
print calculate("I have 3 books and 4 pens. I got 1 more book. How many books do I have now?")
print calculate("Mary is baking a cake . The recipe wants 8 cups of flour . She already put in 2 cups . How many cups does she need to add ?")
print calculate("Sara 's high school played 12 games this year . The team won most of their games . They were defeated during 4 games . How many games did they win ?")
print calculate("There were 6 roses in the vase . Mary cut some roses from her flower garden . There are now 16 roses in the vase . How many roses did she cut ?")
print calculate("Joan went to 4 games this year . She went to 9 games last year . How many games did Joan go to in all ?")
print calculate("Tom has 9 balloons Sara has 8 balloons . How many balloons do they have in total ?")
print calculate("There are 4 trees currently in the park . Park workers will plant 6 trees today . How many trees will the park have when the workers are finished ?")
print calculate("Sam had 9 dimes in his bank . He gave 7 dimes to someone. How many dimes does Sam have now ?")
print calculate("Tom had 5 pencils. He placed 3 pencils in the drawer . How many pencils are remaining ?")


--------



/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:12: DeprecationWarning: The StanfordDependencyParser will be deprecated
Please use nltk.parse.corenlp.StanforCoreNLPDependencyParser instead.


I have three books and four pens . How many books do I have now?
{u'pen': [4], u'book': [3]}
verb have
seedtype = have 0
verb have
seedtype = have 0
3

--------

I have 3 books and 4 pens. I gave 1 book to Tom. How many books do I have now?
{u'pen': [4], u'book': [3, 1]}
verb have
seedtype = have 0
verb have
seedtype = have 0
verb have
seedtype = have 0
verb have
seedtype = have 0
verb gave
seedtype = have give -
2

--------

I have 3 books and 4 pens. I got 1 more book. How many books do I have now?
{u'pen': [4], u'book': [3, 1]}
verb have
seedtype = have 0
verb have
seedtype = have 0
verb have
seedtype = have 0
verb have
seedtype = have 0
verb got
seedtype = have get +
4.0

--------

Mary is baking a cake . The recipe wants 8 cups of flour . She already put in 2 cups . How many cups does she need to add ?
{u'cup': [8, 2]}
6

--------

Sara 's high school played 12 games this year . The team won most of their games . They were defeated during 4 games . How many games did they win ?
{u